In [5]:
%load_ext autoreload
%autoreload 2
import os
import copy
import pickle
import pandas as pd
import copy
from statistics import mean

import numpy as np
import scipy
from tqdm import tqdm

from gensim.models import Word2Vec
from gensim import utils
from gensim.corpora import Dictionary
from gensim.models.lsimodel import LsiModel
from sklearn.preprocessing import MinMaxScaler

import sys
sys.path.append('../src')
from models import get_matrices_from_term_lists, produce_2ndorder_effect_size, produce_1storder_effect_size_unnormalized
from models import produce_2ndorder_p_value, produce_1storder_p_value


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
we_model_name = "sg_dim300_min100_win5"
we_vector_size = 300
we_model_dir = '../data/external/wiki-english/wiki-english-20171001/%s' % we_model_name

In [4]:
we_model = Word2Vec.load(we_model_dir+'/model.gensim')

print ('loading done!')

loading done!


In [8]:
#loading E_ctx_vec
with open(we_model_dir+'/E_ctx_vec.pkl', 'rb') as fr:
    E_ctx_vec = pickle.load(fr)
with open(we_model_dir+'/E_wrd_vec.pkl', 'rb') as fr:
    E_wrd_vec = pickle.load(fr)

# Calculating *d*

In [6]:
# WEAT 6
X_terms = ['John', 'Paul','Mike','Kevin','Steve','Greg','Jeff','Bill']
Y_terms = ['Amy','Joan','Lisa','Sarah','Diana','Kate','Ann','Donna']
A_terms = ['executive','management','professional','corporation',
               'salary','office','business','career']
B_terms = ['home','parents','children','family',
               'cousins','marriage','wedding','relatives']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [13]:
# WEAT 7 using Google News word2vec model
X_terms = ['math','algebra','geometry','calculus',
             'equations','computation','numbers','addition']
Y_terms = ['poetry','art','dance','literature',
             'novel','symphony','drama','sculpture']
A_terms = ['male','man','boy','brother',
              'he','him','his','son']
B_terms = ['female','woman','girl','sister',
               'she','her','hers','daughter']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [33]:
# Weat 8
A_terms = ['brother','father','uncle','grandfather',
          'son','he','his','him']
B_terms = ['sister','mother','aunt','grandmother',
          'daughter','she','hers','her']
X_terms = ['science','technology','physics','chemistry',
          'Einstein','NASA','experiment','astronomy']
Y_terms = ['poetry','art','Shakespeare','dance',
          'literature','novel','symphony','drama']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [10]:
produce_1storder_effect_size_unnormalized(X_terms, Y_terms, A_terms, B_terms, we_model, E_ctx_vec, E_wrd_vec)

1.815586923924345

# First-Order Experiments

In [11]:
print(produce_1storder_effect_size_unnormalized(X_terms, Y_terms, A_terms, B_terms, we_model, E_ctx_vec, E_wrd_vec))
print(X_terms)

1.815586923924345
['john', 'paul', 'mike', 'kevin', 'steve', 'greg', 'jeff', 'bill']


In [ ]:
print(produce_1storder_p_value(we_model, X_terms, Y_terms, A_terms, B_terms, E_ctx_vec, E_wrd_vec))
print(X_terms)

  0%|                                                                            | 9/12870.0 [00:12<5:07:27,  1.43s/it]

# Second-Order

In [26]:
produce_effect_size(we_model, X_terms, Y_terms, A_terms, B_terms)

1.5948170825435086

In [14]:
print(produce_2ndorder_p_value(we_model, X_terms, Y_terms, A_terms, B_terms))
print(X_terms)

100%|██████████████████████████████████████████████████████████████████████████| 12870/12870.0 [08:53<00:00, 24.12it/s]


0.0007125704370340991
['math', 'algebra', 'geometry', 'calculus', 'equations', 'computation', 'numbers', 'addition']


Results: 1.774, 1.595, 1.4183

# Normalized

In [78]:
def get_1storder_association_metric_scaled(word, A_terms, B_terms):
    A_relations = get_expSG_1storder_relation_scaled(word, A_terms)
    B_relations = get_expSG_1storder_relation_scaled(word, B_terms)
    return mean(A_relations.values()) - mean(B_relations.values())

In [92]:
def produce_1storder_effect_size_scaled(X_terms, Y_terms, A_terms, B_terms):
    x_associations = np.array([])
    y_associations = np.array([])
    for (x,y) in zip(X_terms, Y_terms):
        x_association = get_1storder_association_metric_scaled(x, A_terms, B_terms)
        y_association = get_1storder_association_metric_scaled(y, A_terms, B_terms)
        x_associations = np.append(x_associations, x_association)
        y_associations = np.append(y_associations, y_association)
    all_associations = np.append(x_associations, y_associations)
    return (np.mean(x_associations) - np.mean(y_associations))/np.std(all_associations, ddof=1)
produce_1storder_effect_size_scaled(X_terms, Y_terms, A_terms, B_terms)

1.4517385120046582

(2nd order, 1st order unscaled, 1st order scaled)
(1.774, 1.816, 1.759)
(1.595, 1.648, 1.335)
(1.4183, 1.471, 1.451)